In [1]:
import sys
if not '..' in sys.path:
    sys.path.append('..')

In [2]:
from analysis import collection, analysis

from pathlib import Path

import numpy as np
import scipy
from analysis import analysis
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.collections import LineCollection
from matplotlib.colors import to_rgba
from IPython.display import HTML

from matplotlib.gridspec import GridSpec

import matplotlib as mpl
mpl.rcParams.update({'axes.labelsize': 16})

%matplotlib inline

image_path = Path("/Users/spencer/phd/phd/images/")

In [5]:
collection_name = "emg_olympics"
task_name = "center_hold"
c = collection.Collection(collection_name)
subjects = c.alphabetically_sorted_subjects()

In [ ]:
def count_nan(data):
    return np.count_nonzero(np.isnan(data))

max_traj_len = 0
for subject in subjects:
    for session in subject.tasks[task_name].sorted_sessions():
        for trial in session.sorted_trials():
            hold_steps = int(200*trial.hold_time)
            traj = trial.get_trajectory()[hold_steps:,:]
            if traj.shape[0] > max_len:
                max_len = traj.shape[0]
print("Max trajectory length: " , max_len)

x = np.arange(0,max_len)
interp_traj_stack = np.empty(shape=(len(subjects), 15, 12, max_len, 2))
interp_traj_stack[:] = np.nan
for subject_idx, subject in enumerate(subjects):
    for session_idx, session in enumerate(subject.tasks[task_name].sorted_sessions()[:15]):
        for target_idx in range(1,13):
            for trial_idx, trial in enumerate(session.sorted_trials()):
                if trial.outcome != "No Hold" and trial.target_number == target_idx:
                    hold_steps = int(200*trial.hold_time)
                    traj = trial.get_trajectory()[hold_steps:,:]
                    if traj.shape[0] < max_len:
                        interp_traj_stack[subject_idx, session_idx, target_idx-1, :, :] = interp_trajectory(x,traj[:,:])
                    else:
                        interp_traj_stack[subject_idx, session_idx, target_idx-1, :, :] = traj[:,:]

print(interp_traj_stack.shape)
print(count_nan(interp_traj_stack))